### IV.3. Make a program to compute the MLE of the bivariate (so, dimension 2) Non-Central Student t (NCT).
 Recall the "preparation" notes above. It inputs obviously a T X 2 data set, and outputs the MLE parameter vector.	

 The code is an implementation of the Maximum Likelihood Estimation (MLE) method for fitting a bivariate non-central Student's t-distribution to a given dataset. The distribution is characterized by parameters such as degrees of freedom, mean, covariance matrix, and non-centrality parameters. The code employs the L-BFGS-B optimization algorithm to find the MLE of the parameters.

In [20]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import multivariate_t
from scipy.special import gamma, gammaln


In [19]:
''' 
Constrains the input parameters based on the provided lower and upper bounds and updates the covariance matrix accordingly.
Inputs:
    param: Array of parameters to be constrained.
    bound_lo: Array of lower bounds for the parameters.
    bound_hi: Array of upper bounds for the parameters.
    bound_which: Binary array indicating which parameters are subject to bounding.
    V: Optional covariance matrix.
Outputs:
    param: Constrained parameter array.
    V: Updated covariance matrix if provided.
'''

def einschrk(param, bound_lo, bound_hi, bound_which, V=None):
    param = np.asarray(param)
    bound_lo = np.asarray(bound_lo)
    bound_hi = np.asarray(bound_hi)
    bound_which = np.asarray(bound_which)

    if V is not None:
        V = np.asarray(V)

    param[bound_which == 1] = np.maximum(param[bound_which == 1], bound_lo[bound_which == 1])
    param[bound_which == 1] = np.minimum(param[bound_which == 1], bound_hi[bound_which == 1])

    if V is not None:
        V = np.dot(np.dot(np.diag(bound_which), V), np.diag(bound_which))

    return param, V


In [18]:
'''
Computes the pdf of the bivariate non-central Student's t-distribution for the given data and parameters.
Inputs:
    x: Data array.
    df: Degrees of freedom.
    mu: Mean vector.
    Sigma: Covariance matrix.
    ncp: Non-centrality parameters.
Output:
    y: Probability density function (pdf) value.
'''

def mvtpdfmine(x, df, mu, Sigma, ncp):
    d = len(x)
    mu = np.zeros(d) if mu is None else mu
    Sigma = np.eye(d) if Sigma is None else Sigma

    term = np.sum((x - mu) * np.linalg.solve(Sigma, (x - mu))) + np.sum((ncp * np.linalg.solve(Sigma, ncp)) / df)

    logN = -((df + d) / 2) * np.log(1 + term / df)
    logD = 0.5 * np.log(np.linalg.det(Sigma)) + (d / 2) * np.log(df * np.pi)
    y = np.exp(gammaln((df + d) / 2) - gammaln(df / 2) + logN - logD)

    return y

$$ 
\left(1 + \frac{\nu_1}{\nu_1 + \nu_2} \left( \frac{\nu_1}{\nu_1 + \nu_2} \Sigma \right) (x - \delta_1)^2 + \frac{\nu_2}{\nu_1 + \nu_2} \left( \frac{\nu_2}{\nu_1 + \nu_2} \Sigma \right) (y - \delta_2)^2 \right)^{-\frac{2\nu_1 + \nu_2}{2} - \frac{1}{2}}
-2 \nu_1 + \nu_2
$$


In [69]:
'''
Trying another way to compute the pdf of the bivariate non-central Student's t-distribution for the given data and parameters.
''' 

def bvnct_pdf(x, df, ncp, Sigma):
    delta = ncp
    d = len(x)
    nu = df / 2

    term_x = (x[0] - delta[0]) / np.sqrt((nu[0] / np.sum(nu)) * Sigma[0, 0])
    term_y = (x[1] - delta[1]) / np.sqrt((nu[1] / np.sum(nu)) * Sigma[1, 1])

    term = term_x**2 + term_y**2
    constant = 1 / (2 * np.pi * np.sqrt(np.linalg.det(Sigma)))
    gamma_term = np.exp(gammaln(np.sum(nu) / 2 + 0.5) - np.sum(gammaln(nu / 2)))

    pdf = constant * gamma_term * (1 + term / np.sum(nu))**(-np.sum(nu) / 2 - 0.5)

    return pdf

# Example usage:
#x = np.array([[1.0, 3.0], [2.0, 4.0]])  # Each column corresponds to a variable
x = data
df = np.array([3, 4])
ncp = np.array([0.0, 0.0])
Sigma = np.array([[1.0, 0.0], [0.0, 2.0]])  # Assuming independent variables, so covariance is 0

pdf_matrix_result = bvnct_pdf_matrix(x, df, ncp, Sigma)
print(pdf_matrix_result)

#pdf = np.array([bvnct_pdf(x, df, ncp, Sigma) for xi in x])
#print(pdf)

[0.00356262 0.07034922]


In [83]:


def mvtnct_logpdf(x, df, mu, Sigma, ncp):
    d = len(x)
    mu = mu.reshape((d, 1))
    ncp = ncp.reshape((d, 1))

    term = (x - mu).T @ np.linalg.inv(Sigma) @ (x - mu)
    non_central_term = ncp.T @ np.linalg.inv(Sigma) @ (x - mu) / df

    log_N = -((df + d) / 2) * np.log(1 + term / df)
    log_D = 0.5 * np.log(np.linalg.det(Sigma)) + (d / 2) * np.log(df * np.pi)

    log_pdf = log_N + log_D + non_central_term

    return log_pdf

# Example usage:
x_example = np.array([0.2, 0.3])  # Replace with your own values
df_example = 2
mu_example = np.zeros(2)  # Replace with your own values
Sigma_example = np.eye(2)  # Replace with your own values
ncp_example = np.array([1.0, 1.5])  # Replace with your own values

log_pdf_result = mvtnct_logpdf(x_example, df_example, mu_example, Sigma_example, ncp_example)
print(log_pdf_result)


[[2.00943564 2.09633925]
 [1.97133925 2.04052167]]


In [84]:


def mvnctpdfln(x, mu, ncp, v, Sigma):
    # x: d X T matrix of evaluation points
    # mu, gam: d-length location and noncentrality vector
    # v: degrees of freedom
    # Sigma: dispersion matrix
    gam = ncp
    d, t = x.shape
    C = Sigma
    R, err = np.linalg.cholesky(C, lower=True, check_finite=False)
    
    assert err == 0, 'C is not (semi) positive definite'
    
    mu = np.reshape(mu, (len(mu), 1))
    gam = np.reshape(gam, (len(gam), 1))
    
    vn2 = (v + d) / 2
    xm = x - np.tile(mu, (1, t))
    rho = np.sum(np.linalg.solve(R.T, xm)**2, axis=0)
    
    pdfln = np.log(np.abs(np.prod(np.diag(R)))) - (d / 2) * np.log(np.pi * v) - \
            np.log(np.abs(gamma(v / 2))) - np.sum(np.log(np.abs(np.diag(R)))) - vn2 * np.log1p(rho / v)
    
    if np.all(gam == 0):
        return pdfln
    
    idx = (pdfln >= -37)
    maxiter = int(1e4)
    k = 0
    
    if np.any(idx):
        gcg = np.sum((np.linalg.solve(R.T, gam))**2)
        pdfln = pdfln - 0.5 * gcg
        xcg = np.dot(xm.T, np.linalg.solve(C, gam))
        term = 0.5 * np.log(2) + np.log(xcg) - 0.5 * slog(v + rho)
        
        term[term == -np.inf] = np.log(np.finfo(float).tiny)
        term[term == np.inf] = np.log(np.finfo(float).max)
        
        logterms = gammaln((v + d + k) / 2) - gammaln(k + 1) - gammaln(vn2) + k * term
        ff = np.exp(logterms)
        logsumk = np.log(ff)
        
        while k < maxiter:
            k = k + 1
            logterms = gammaln((v + d + k) / 2) - gammaln(k + 1) - gammaln(vn2) + k * term[idx]
            ff = np.exp(logterms - logsumk[idx])
            logsumk[idx] = logsumk[idx] + np.log1p(ff)
            idx[idx] = (np.abs(ff) > 1e-4)
            
            if not np.any(idx):
                break
    
    pdfln = pdfln + logsumk
    
    return pdfln

def slog(x):
    # Truncated log. No -Inf or +Inf.
    return np.log(np.maximum(np.finfo(float).tiny, np.minimum(np.finfo(float).max, x)))


In [48]:
'''
Computes the log-likelihood of the bivariate non-central Student's t-distribution given the parameters and data.
Inputs:
    param: Array of parameters.
    x: Data array.
Output:
    ll: Log-likelihood value.
'''

def MVTloglik(param, x):
    nobs, d = x.shape
    k = param[0]
    mu = param[1:3]
    ncp = param[6:8]
    
    Sig = np.zeros((d, d))
    Sig[0, 0] = param[3]
    Sig[1, 1] = param[5]
    Sig[0, 1] = param[4]
    Sig[1, 0] = Sig[0, 1]

    if np.min(np.linalg.eigvals(Sig)) < 1e-10:
        ll = 1e5
    else:
        #pdf = np.array([mvtpdfmine(xi, k, mu, Sig, ncp) for xi in x])
        pdf = np.array([bvnct_pdf(x, df, ncp, Sigma) for xi in x])
        llvec = np.log(pdf)
        ll = -np.mean(llvec)
        if np.isinf(ll):
            ll = 1e5

    return ll

In [70]:

MVTloglik(param, x)

4.145771118986263

In [86]:
'''
Estimates the parameters of the bivariate non-central Student's t-distribution using the Maximum Likelihood Estimation method. 
It employs the L-BFGS-B optimization algorithm to maximize the log-likelihood (Actually minimizing the negative log-likelihood). 
The results include the estimated parameters, standard errors, number of iterations, log-likelihood, and the covariance matrix of the estimated parameters.

Inputs:
    x: Data array.
    initvec: Initial guess for the parameters.
Outputs:
    param: Estimated parameters.
    stderr: Standard errors of the parameters.
    iters: Number of iterations performed during optimization.
    loglik: Log-likelihood of the estimated parameters.
    Varcov: Covariance matrix of the estimated parameters.
'''

def MVTestimation(initvec, x):
    nobs, d = x.shape
    if d != 2:
        raise ValueError('not done yet, use EM')

    if d == 2:
        # k mu1 mu2 s11 s12 s22 ncp1 ncp2
        bound_lo = [0.2, -1, -1, 0.01, -90, 0.01, -10, -10]
        bound_hi = [20, 1, 1, 90, 90, 90, 10, 10]
        bound_which = [1, 0, 0, 1, 1, 1, 1, 1]
        

    maxiter = 300
    tol = 1e-7
    MaxFunEvals = len(initvec) * maxiter

    bounds = tuple((lo, hi) for lo, hi in zip(bound_lo, bound_hi) if any(bound_which))

    result = minimize(MVTloglik, initvec, args=(x,), bounds=bounds, method='L-BFGS-B',
                      options={'disp': True, 'maxiter': maxiter, 'ftol': tol, 'maxfun': MaxFunEvals})

    pout = result.x
    fval = result.fun
    hess = result.hess_inv.todense()
    nobs = len(x)
    
    V = np.linalg.inv(hess) / nobs
    #param, V = einschrk(pout, bound_lo, bound_hi, V)
    param, V = einschrk(pout, bound_lo, bound_hi, bound_which, V)
    param = param.reshape(-1, 1)
    Varcov = V
    stderr = np.sqrt(np.diag(V))
    loglik = -fval * nobs
    iters = result.nit

    return param.flatten(), stderr.flatten(), iters, loglik, Varcov


: 

Testing the function and estimating MLE param from our "home-made" data-set

In [76]:
nobs, d = data.shape

# k mu1 mu2 s11 s12 s22 ncp1 ncp2
bound_lo = [0.2, -1, -1, 0.01, -90, 0.01, -10, -10]
bound_hi = [20, 1, 1, 90, 90, 90, 10, 10]
bound_which = [1, 0, 0, 1, 1, 1, 1, 1]
        

maxiter = 300
tol = 1e-7
MaxFunEvals = len(initvec) * maxiter
bounds = tuple((lo, hi) for lo, hi in zip(bound_lo, bound_hi) if any(bound_which))

result = minimize(MVTloglik, initvec, args=(x,), bounds=bounds, method='L-BFGS-B',
         options={'disp': True, 'maxiter': maxiter, 'ftol': tol, 'maxfun': MaxFunEvals})


pout = result.x
fval = result.fun
hess = result.hess_inv.todense()
nobs = len(x)


In [77]:
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 4.124929318453386
        x: [ 2.000e+00 -8.000e-01 -2.000e-01  2.000e+01  2.000e+00
             1.000e+01  2.291e-01  7.477e-02]
      nit: 6
      jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
             0.000e+00  8.971e-06 -2.647e-05]
     nfev: 63
     njev: 7
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>

In [67]:
MVTestimation(initvec, data)

TypeError: 'float' object is not subscriptable

In [63]:
# Test the function
param, stderr, iters, loglik, Varcov = MVTestimation(initvec, data)
print('Estimated Parameters:')
print(param)
print('Standard Errors:')
print(stderr)
print('Log-Likelihood:', loglik)
print('Number of Iterations:', iters)

TypeError: 'float' object is not subscriptable

### IV.3 cont. 
Now make a program that inputs a T X 2 data set, and computes and outputs the MLE of the bivariate 2-component mixture Laplace, and that of the bivariate NCT. This is obviously trivial --- it just combines your previous two programs. What is new is this: You also output the AIC and BIC values corresponding to to each of the two models, so that we can, via these in-sample model-comparison methods, ascertain which model is preferred by the data.

In [10]:
import matlab.engine
#eng = matlab.engine.start_matlab("-r 'format long'")



ModuleNotFoundError: No module named 'matlab'

In [ ]:
eng = matlab.engine.start_matlab("-r 'format long'")
params, stderr = np.array(eng.MixLapEstimate(Y, initguess, nargout=2))

The MixLapEstimate.m function requires 2 inputs, Y (n, 2), init_guess (13, )